# Scipp widgets

[Scipp](https://scipp.github.io/) is a python data structure module for handling multi-dimensional data arrays with labeled dimensions. 
Data reduction or analysis workflows using this module often take the form of ipython notebooks. 


Scipp-widgets is a seperate module containing some simple scipp specific wrappers using [ipywidgets](https://ipywidgets.readthedocs.io/) to allows graphical elements to be easily incorprated into such notebooks.
This can be installed using conda with the command `conda install scipp_widgets ---channel scipp/label/dev`

In this context a widget is a generic term describing the building blocks from which a user interface can be constructed. As widgets are often nested this can lead to the entire interface being a widget which contains sub-widgets within it.

## Introductory examples
For demonstration purposes we first create a simple set of [scipp](https://scipp.github.io/tutorials/multi-d-datasets.html) data.

In [ ]:
import numpy as np
import scipp as sc
var =sc.Variable(['x', 'z'], values=np.arange(0.0, 10.0, 0.1).reshape(10, 10),
                                              variances=0.1*np.random.rand(10, 10))
sc.show(var)

## Wrapping a processing function
scipp-widgets allows functions to be wrapped in simple graphical interfaces.
Taking inputs from, and returning outputs to, the notebook's global scope.
As a first example, take the `scipp.sum` function.

We will need to create an input specification object describing how we want the graphical inputs displayed, what validation we wish to perform and which argument of the underlying function they should correspond to. Currently there is no way to specify layout options.

In [ ]:
from scipp_widgets.input_spec import InputSpec, TextInputSpec
data_input = InputSpec('x', tooltip='input data')
dimension_input = TextInputSpec('dim', options = ('x', 'z'))

A simeple user interface which wraps the sum function can then be constructed like so:

In [ ]:
from scipp_widgets.widgets import ProcessWidget
ProcessWidget(sc.sum, [data_input, dimension_input])

## Wrapping a display function

Another common case is to wrap a function and display its result without adding it to the notebooks global scope.
As an example, take the `scipp.plot.plot` function. This takes a scipp object and plots it. 

In [ ]:
from scipp_widgets.widgets import DisplayWidget
data_input = InputSpec('scipp_obj')
DisplayWidget(sc.plot.plot, [data_input])

## Validators
Validator methods can be added to any input. 
These are run when the widget is processing and serve to both perform any pre-processing the input needs before it is passed to the underlying function as well as perform any validation specified.

For example we could add a validator to our wrapped plot function meaning it will only accept scipp objects as inputs.

In [ ]:
from scipp_widgets.validators import scipp_object_validator
data_input = InputSpec('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input])

## Hiding code
The code block used to create a widget can be hidden using the key word argument `hide_code`. This adds an additional button loabelled `Py` to toggle the visability of the preceding code block.

In [ ]:
data_input = InputSpec('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input], hide_code=True)